In [1]:
!pip install peft

In [2]:
!pip install tokenizers

In [3]:
!pip install wandb

In [1]:
!pip install sacrebleu rouge_score

  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/cd/45/7b55a7bd7e5c5b573b40ad58ba43fa09962dc5c8d71b1f573d4aeaa54a7e/sacrebleu-2.5.1-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/51.8 kB ? eta -:--:--
     ------- -------------------------------- 10.2/51.8 kB ? eta -:--:--
     ------- -------------------------------- 10.2/51.8 kB ? eta -:--:--
     --------------- ---------------------- 20.5/51.8 kB 131.3 kB/s eta 0:00:01
     ---------------------- --------------- 30.7/51.8 kB 163.8 kB/s eta 0:00:01
     -------------------------------------  51.2/51.8 kB 219.0 kB/s eta 0:00:01
     -------------------------------------- 51.8/51.8 kB 189.9 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): star

In [18]:
!pip install trl

^C


In [21]:
!pip install --upgrade transformers

In [1]:
from transformers import AutoTokenizer, AutoConfig, Gemma2ForCausalLM 

# AutoModelForCausalLM,TrainingArguments, Trainer
from trl import SFTTrainer, SFTConfig
from trl.trainer import ConstantLengthDataset
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import bitsandbytes as bnb
import wandb
import torch
import os
from scripts.vandb_callbacks import WandbMetricsCallback
from sacrebleu import corpus_bleu
from rouge_score import rouge_scorer
from datetime import datetime  

In [20]:
os.chdir(r"C:\Users\Viktor\Documents\TMS\NLP\NLP_psyhology_support")
tokenizer_save_path = r"models\gemma2_russian_tokenizer"
dataset_path = "Egor-AI/Dataset_of_Russian_thinking"
model_name = "google/gemma-2-2b"
output_dir = r"models\gemma2-russian"
wandb_key = 'b0c232d05724b649f39a0608ff0b3d856890f922'
run_name='gemma2-ru_v1'+str(datetime.now())
max_seq_length = 512

In [3]:
wandb.login(key=wandb_key)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Viktor\_netrc
wandb: Currently logged in as: vopetrashkevich (vopetrashkevich-individual-activity) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [4]:
wandb.init(
    project="gemma2_russian",
    # Track hyperparameters and run metadata.
    config={
        "learning_rate": 1e-4,
        "architecture": "gemma2",
        "dataset": dataset_path,
        "epochs": 3,
    },
)

In [5]:
# --- Tokenizer Loading ---
# Correctly load the tokenizer from the saved directory
loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_path)
loaded_tokenizer_backend = loaded_tokenizer.backend_tokenizer
# Now we will load our tokenizer that saved in correct folder, with correct files, like config.json
loaded_tokenizer.pad_token = loaded_tokenizer.eos_token  # Установка padding-токена

# Загрузка датасета
# dataset_ru_dialog = load_dataset("Den4ikAI/russian_dialogues")
dataset_ru_thinking = load_dataset(dataset_path, split="train")

In [6]:
# Функция преобразования
def preprocess_entry(examples):
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    examples['text'] = template.format(instruction=examples["system"]+examples["prompt"], response=examples["response"])
    return examples
 

In [7]:
def formatting_func(example):
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    example['text'] = template.format(instruction=example['Question'], response=example['Answer'])
    return example

In [8]:
def ds_preprocessing(dataset):
    return  dataset.map(preprocess_entry, remove_columns=dataset.column_names)

In [9]:
dataset = ds_preprocessing(dataset_ru_thinking)

In [10]:
dataset[:5]

{'text': ['Instruction:\nYou are an AI assistant. Format your answers as follows: <Thought> Your thoughts (understanding, reasoning) </Thought> <output> Your answer </output>Свежеокрашенная круглая эмблема на футбольном поле полностью покрыта брезентом минимально возможного квадратного размера. Брезент занимает площадь 196 квадратных футов. Какова площадь круглой эмблемы в квадратных футах? Выразите свой ответ через $\\pi$.\n\nResponse:\n<Thought>Итак, у меня есть эта проблема. На футбольном поле стоит свежевыкрашенная круглая эмблема, полностью накрытая минимально квадратным брезентом. Брезент покрывает площадь 196 квадратных футов, и мне нужно найти площадь круглой эмблемы через число π.\n\nХорошо, давайте начнем с визуализации этого. Вот круг, накрытый квадратным брезентом. Проблема в том, что он накрыт брезентом минимально возможной квадратной формы. Итак, это означает, что квадрат идеально вписывается в круг, верно? Другими словами, круг вписан в квадрат.\n\nПодождите, если круг в

In [11]:
# Разбиваем на тренировочную (80%) и тест+валидацию (20%)
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Разбиваем тест+валидацию ещё раз на тестовую (10%) и валидационную (10%)
test_valid_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)

# # Собираем новый датасет
# dataset_splits = {
#     "train": train_test_split["train"],
#     "validation": test_valid_split["train"],
#     "test": test_valid_split["test"]
# }

In [12]:
train_dataset = ConstantLengthDataset(
    dataset=train_test_split["train"],
    tokenizer=loaded_tokenizer,
    seq_length=max_seq_length,  # Длина последовательности в токенах
    dataset_text_field="text",
    # shuffle=True,    # Перемешивать данные
    # append_tail=True # Добавить остатки последовательностей
)

test_dataset = ConstantLengthDataset(
    dataset=test_valid_split["train"],
    tokenizer=loaded_tokenizer,
    seq_length=max_seq_length,  # Длина последовательности в токенах
    dataset_text_field="text"
)

valid_dataset = ConstantLengthDataset(
    dataset=test_valid_split["test"],
    tokenizer=loaded_tokenizer,
    seq_length=max_seq_length,  # Длина последовательности в токенах
    dataset_text_field="text"
)

In [13]:
# Разбиваем на тренировочную (80%) и тест+валидацию (20%)
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

# Разбиваем тест+валидацию ещё раз на тестовую (10%) и валидационную (10%)
test_valid_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)

# Собираем новый датасет
dataset_splits = {
    "train": train_test_split["train"],
    "validation": test_valid_split["train"],
    "test": test_valid_split["test"]
}

In [14]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

In [24]:

config = AutoConfig.from_pretrained(model_name, use_cash=False)
model = Gemma2ForCausalLM.from_pretrained(model_name, attn_implementation="eager", torch_dtype=torch.float32, device_map='cpu')
model.resize_token_embeddings(len(loaded_tokenizer))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(256212, 2304, padding_idx=0)

In [25]:
class GenerationMetricsCallback(WandbMetricsCallback):
    def __init__(self, tokenizer, eval_dataset, max_length=max_seq_length):
        """
        Инициализация callback для метрик генерации.

        Args:
            tokenizer: Токенизатор модели.
            eval_dataset: Валидационный датасет.
            max_length: Максимальная длина генерируемого текста.
        """
        super().__init__()  # Инициализация базового класса WandbMetricsCallback
        self.tokenizer = tokenizer
        self.eval_dataset = eval_dataset
        self.max_length = max_length
        self.rouge_scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    def on_evaluate(self, args, state, control, model=None, **kwargs):
        """
        Метод, вызываемый во время оценки модели. Вычисляет BLEU и ROUGE.
        """
        if model is None:
            return

        predictions = []
        references = []

        # Генерация текста для каждого примера в валидационном наборе
        for example in self.eval_dataset:
            input_ids = example['input_ids'].unsqueeze(0)  # Добавляем batch dimension
            generated_ids = model.generate(input_ids, max_length=self.max_length)
            generated_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            reference_text = self.tokenizer.decode(example['labels'], skip_special_tokens=True)
            predictions.append(generated_text)
            references.append(reference_text)

        # Расчет BLEU
        bleu_score = corpus_bleu(predictions, [references]).score

        # Расчет ROUGE
        rouge_scores = {'rouge1': 0, 'rouge2': 0, 'rougeL': 0}
        for pred, ref in zip(predictions, references):
            scores = self.rouge_scorer.score(ref, pred)
            for key in rouge_scores:
                rouge_scores[key] += scores[key].fmeasure
        for key in rouge_scores:
            rouge_scores[key] /= len(predictions)

        # Логирование метрик в wandb
        wandb.log({
            "eval_bleu": bleu_score,
            "eval_rouge1": rouge_scores['rouge1'],
            "eval_rouge2": rouge_scores['rouge2'],
            "eval_rougeL": rouge_scores['rougeL'],
        })

In [26]:
# torch._dynamo.config.suppress_errors = True
# model = torch.compile(model)

model = get_peft_model(model, peft_config)

In [27]:
generation_callback = GenerationMetricsCallback(
    tokenizer=loaded_tokenizer,  # Ваш токенизатор
    eval_dataset=valid_dataset,  # Ваш валидационный датасет
    max_length=max_seq_length  # Максимальная длина генерируемого текста
)

In [28]:
# training_args = TrainingArguments(
#     output_dir="models/gemma2-russian",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     run_name='gemma2-ru_v1',
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=4,
#     gradient_checkpointing=True,
#     gradient_accumulation_steps=4,
#     learning_rate=1e-4,
#     num_train_epochs=3,
#     save_total_limit=2,
#     logging_dir="./logs",
#     logging_steps=10,
#     report_to="wandb",
#     fp16=False,
#     bf16=False,
#     # use_cpu=True,
# )

ftp_training_args = SFTConfig(
    output_dir=output_dir,
    run_name=run_name,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    logging_dir="./logs",
    report_to="wandb",
    optim="adamw_torch", 
    save_steps=0,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    },)

In [29]:
for param in model.parameters():
    param.requires_grad = True

trainer = SFTTrainer(
    model=model,
    args=ftp_training_args,
    train_dataset=train_dataset,
    peft_config=peft_config,
    eval_dataset=valid_dataset,
    callbacks=[generation_callback],
)


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

trainer.evaluate(eval_dataset=test_dataset)
# Сохранение модели
model.save_pretrained("models/gemma2-russian")
loaded_tokenizer.save_pretrained("models/gemma2-russian")

In [ ]:
print('hello world')

In [1]:
print('hello world')

hello world
